# CS-C3240 - Machine Learning D
## Project
**Author: Aaron Gutierrez-Hernandez & Alexandre Cojot**

**Date created: 10-sep-2023**

**Last modified: 10-sep-2023**

## Import Libraries and Data

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Read the .csv file containing the dataset
data = pd.read_csv('heart_data.csv')
data.head()

## Clean Data

In [ ]:
data.drop_duplicates(inplace=True)
data.info()

## EDA

In [ ]:
data.describe()

## Prepare Data

In [ ]:
X = data.drop('target',axis=1) # Split the features from the labels
y = data['target']             # Split the labels from the features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # Split the dataset into train and test sets

## Create ML Model